In [1]:
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, BatchNormalization, ReLU, Conv2D, Dense, MaxPool2D, AvgPool2D, GlobalAvgPool2D, Concatenate

In [2]:
def bn_relu_conv(x, filters, kernel_size):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters, kernel_size, padding="same")(x)
    return x

In [3]:
def dense_block(tensor, k, reps):
    for _ in range(reps):
        x = bn_relu_conv(tensor, 4*k, 1)
        x = bn_relu_conv(x, k, 3)
        tensor = Concatenate()([tensor, x])
    return tensor

In [4]:
def transition_layer(x, theta):
    f = int(tensorflow.keras.backend.int_shape(x)[-1] * theta)
    x = bn_relu_conv(x, f, 1)
    x = AvgPool2D(2, strides=2, padding="same")(x)
    return x

In [5]:
k = 48
theta = 0.5
repetitions = [6, 12, 36, 24]

input = Input(shape=(224,224,3))

x = Conv2D(2*k, 7, strides=2, padding="same")(input)
x = MaxPool2D(3, strides=2, padding="same")(x)

for reps in repetitions:
    d = dense_block(x, k, reps)
    x = transition_layer(d, theta)
    
x = GlobalAvgPool2D()(d)
output = Dense(62, activation="softmax")(x)


model = Model(input, output)

In [6]:

#plot_model(model, show_shapes=True)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.526225 to fit

